In [ ]:
mua = acr.mua.load_concat_peaks_df(subject, exp)
mua = mua.sort('datetime')
lfp = acr.io.load_concat_raw_data(subject, ['swi-bl', 'swi'])

In [ ]:
#-------------------------- Standard Imports --------------------------#
%reload_ext autoreload
%autoreload 2
import kdephys as kde
import pandas as pd
import polars as pl
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import acr
import xarray as xr
from scipy import stats
# ---------------------------- EXTRAS --------------------------------#
from kdephys.plot.main import _title, bp_plot
import kdephys.utils.spectral as sp
bands = sp.bands

import warnings
warnings.filterwarnings('ignore')
import matplotlib as mpl
from acr.utils import *
pu = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/pub_utils.py', 'pu')
dag = acr.utils.import_publication_functions('/home/kdriessen/gh_master/PUBLICATION__ACR/data_agg.py', 'dag')

In [ ]:
subject = 'ACR_25'
exp = 'swi'

In [ ]:
mua = acr.mua.load_concat_peaks_df(subject, exp)

In [ ]:
pon, poff = acr.stim.get_individual_pulse_times(subject, exp)

In [ ]:
emg = acr.io.load_concat_raw_data(subject, ['swi-bl', 'swi'], stores=['EMGr'])
fp = acr.io.load_concat_raw_data(subject, ['swi-bl', 'swi'])

In [ ]:
sd_true_start, stim_start, stim_end, reb_start, full_exp_start = acr.info_pipeline.get_sd_exp_landmarks(subject, exp, update=True)
h = acr.io.load_hypno_full_exp(subject, exp)
hd = acr.hypnogram_utils.create_acr_hyp_dict(subject, exp)

In [ ]:
oodf = dag.compute_hybrid_off_df(subject, exp, chan_threshold=12)
oodf = acr.oo_utils.enhance_oodf(oodf, h, hd)

In [ ]:
plt.rcdefaults()
acr.plots.lrg()
for pulse in np.arange(2500, 2720, 15):
    t1 = pon[pulse]
    t2 = t1 + pd.Timedelta(seconds=5)
    # plot the optrode:
    lfp_dat = fp.sel(datetime=slice(t1, t2)).prb('NNXo').data.T
    lfp_times = fp.sel(datetime=slice(t1, t2)).prb('NNXo').datetime.values
    mua_dat = mua.ts(t1, t2).prb('NNXo')
    f, ax = kde.plot.main.plot_lfp_mua_combined(lfp_dat, mua_dat, lfp_times)
    
    for p in pon[pulse:pulse+15]:
        for a in ax:
            a.axvspan(p, p+pd.Timedelta('100ms'), color='red', alpha=0.2)
    
    ax = kde.plot.main.shade_oodf(ax, oodf.prb('NNXo').oots(t1, t2))
    f.suptitle(f'Pulse {pulse}')

In [ ]:
for pulse in [2524]:
    t1 = pon[pulse]
    t2 = t1 + pd.Timedelta(seconds=5)
    # plot the optrode:
    lfp_dat = fp.sel(datetime=slice(t1, t2)).prb('NNXo').data.T
    lfp_times = fp.sel(datetime=slice(t1, t2)).prb('NNXo').datetime.values
    mua_dat = mua.ts(t1, t2).prb('NNXo')
    f, ax = kde.plot.main.plot_lfp_mua_combined(lfp_dat, mua_dat, lfp_times)
    
    for p in pon[pulse:pulse+15]:
        for i, a in enumerate(ax):
            a.axvspan(p, p+pd.Timedelta('100ms'), color='red', alpha=0.2)
            if i != 16:
                a.set_ylim(-795, 785)
    ax = kde.plot.main.shade_oodf(ax, oodf.prb('NNXo').oots(t1, t2))

    f.suptitle(f'Pulse {pulse}')
    fmg, axmg = plt.subplots(1, 1, figsize=(36, 4))
    axmg.plot(emg.prb('EMGr').sel(datetime=slice(t1, t2)).ch(1).datetime, emg.prb('EMGr').sel(datetime=slice(t1, t2)).ch(1).data)

In [ ]:
[pd.Timestamp(lfp_times.min()) + pd.Timedelta(seconds=i) for i in range(5)]

In [ ]:
kde.plot.main.plot_lfp_mua_combined?

In [ ]:
pulse_to_start = 2524
FSIZE = (10, 8)
mua_height_multiplier = 0.1
lw = 3
rsz = 50
seconds = 2

In [ ]:
fig_name = 'ACR_25__swi__OFF_INDUCTION_SCHEMATIC--NNXo-BASE.png'
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = True
plt.rcParams['xtick.major.size'] = 14
for pulse in [pulse_to_start]:
    t1 = pon[pulse] - pd.Timedelta('60ms')
    t2 = t1 + pd.Timedelta(seconds=seconds)
    # plot the optrode:
    lfp_dat = fp.sel(datetime=slice(t1, t2)).prb('NNXo').data.T
    lfp_times = fp.sel(datetime=slice(t1, t2)).prb('NNXo').datetime.values
    mua_dat = mua.ts(t1, t2).prb('NNXo')
    f, ax = kde.plot.main.plot_lfp_mua_combined(lfp_dat, mua_dat, lfp_times, lfp_subplot_hspace=-0.6, lfp_color=SOM_BLUE, mua_color=SOM_BLUE, figsize=FSIZE, mua_height_multiplier=mua_height_multiplier, lw=lw, rsz=rsz)
    xticks = [pd.Timestamp(lfp_times.min()) + pd.Timedelta(seconds=i) for i in range(seconds+1)]
    ax[16].set_xticklabels([])
    ax[16].set_xticks(xticks)
    for i, a in enumerate(ax):
        if i != 16:
            a.set_ylim(-795, 785)
f.savefig(f'{PAPER_FIGURE_ROOT}/OFF_PERIODS/SCHEMATICS/{fig_name}', dpi=600, transparent=True, bbox_inches='tight')

In [ ]:
fig_name = 'ACR_25__swi__OFF_INDUCTION_SCHEMATIC--NNXo-BASE-1000uv.png'
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = True
plt.rcParams['xtick.major.size'] = 14
for pulse in [pulse_to_start]:
    t1 = pon[pulse] - pd.Timedelta('60ms')
    t2 = t1 + pd.Timedelta(seconds=seconds)
    # plot the optrode:
    lfp_dat = fp.sel(datetime=slice(t1, t2)).prb('NNXo').data.T
    lfp_times = fp.sel(datetime=slice(t1, t2)).prb('NNXo').datetime.values
    mua_dat = mua.ts(t1, t2).prb('NNXo')
    f, ax = kde.plot.main.plot_lfp_mua_combined(lfp_dat, mua_dat, lfp_times, lfp_subplot_hspace=-0.6, lfp_color=SOM_BLUE, mua_color=SOM_BLUE, figsize=FSIZE, mua_height_multiplier=mua_height_multiplier, lw=lw, rsz=rsz)
    xticks = [pd.Timestamp(lfp_times.min()) + pd.Timedelta(seconds=i) for i in range(seconds+1)]
    ax[16].set_xticklabels([])
    ax[16].set_xticks(xticks)
    for p in pon[pulse:pulse+12]:
        for i, a in enumerate(ax):
            if i != 16:
                a.set_ylim(-795, 785)
# add vrect to ax[8]
ax[8].vlines(xticks[1], -500, 500, color='green', alpha=0.8, linewidth=8)
f.savefig(f'{PAPER_FIGURE_ROOT}/OFF_PERIODS/SCHEMATICS/{fig_name}', dpi=600, transparent=True, bbox_inches='tight')

In [ ]:
fig_name = 'ACR_25__swi__OFF_INDUCTION_SCHEMATIC--NNXo-STIM_OVERLAYED.png'
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = True
plt.rcParams['xtick.major.size'] = 14
for pulse in [pulse_to_start]:
    t1 = pon[pulse] - pd.Timedelta('60ms')
    t2 = t1 + pd.Timedelta(seconds=seconds)
    # plot the optrode:
    lfp_dat = fp.sel(datetime=slice(t1, t2)).prb('NNXo').data.T
    lfp_times = fp.sel(datetime=slice(t1, t2)).prb('NNXo').datetime.values
    mua_dat = mua.ts(t1, t2).prb('NNXo')
    f, ax = kde.plot.main.plot_lfp_mua_combined(lfp_dat, mua_dat, lfp_times, lfp_subplot_hspace=-0.6, lfp_color=SOM_BLUE, mua_color=SOM_BLUE, figsize=FSIZE, mua_height_multiplier=mua_height_multiplier, lw=lw, rsz=rsz)
    xticks = [pd.Timestamp(lfp_times.min()) + pd.Timedelta(seconds=i) for i in range(seconds+1)]
    ax[16].set_xticklabels([])
    ax[16].set_xticks(xticks)
    for p in pon[pulse:pulse+6]:
        for i, a in enumerate(ax):
            a.axvspan(p, p+pd.Timedelta('100ms'), color='red', alpha=0.2)
            if i != 16:
                a.set_ylim(-795, 785)
f.savefig(f'{PAPER_FIGURE_ROOT}/OFF_PERIODS/SCHEMATICS/{fig_name}', dpi=600, transparent=True, bbox_inches='tight')

In [ ]:
fig_name = 'ACR_25__swi__OFF_INDUCTION_SCHEMATIC--NNXo-OFF_OVERLAYED.png'
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = True
plt.rcParams['xtick.major.size'] = 14
for pulse in [pulse_to_start]:
    t1 = pon[pulse] - pd.Timedelta('60ms')
    t2 = t1 + pd.Timedelta(seconds=seconds)
    # plot the optrode:
    lfp_dat = fp.sel(datetime=slice(t1, t2)).prb('NNXo').data.T
    lfp_times = fp.sel(datetime=slice(t1, t2)).prb('NNXo').datetime.values
    mua_dat = mua.ts(t1, t2).prb('NNXo')
    f, ax = kde.plot.main.plot_lfp_mua_combined(lfp_dat, mua_dat, lfp_times, lfp_subplot_hspace=-0.6, lfp_color=SOM_BLUE, mua_color=SOM_BLUE, figsize=FSIZE, mua_height_multiplier=mua_height_multiplier, lw=lw, rsz=rsz)
    xticks = [pd.Timestamp(lfp_times.min()) + pd.Timedelta(seconds=i) for i in range(seconds+1)]
    ax[16].set_xticklabels([])
    ax[16].set_xticks(xticks)
    for p in pon[pulse:pulse+6]:
        for i, a in enumerate(ax):
            if i != 16:
                a.set_ylim(-795, 785)
    ax = kde.plot.main.shade_oodf([ax[16]], oodf.prb('NNXo').oots(t1, t2), off_color='red', alpha=0.3)

f.savefig(f'{PAPER_FIGURE_ROOT}/OFF_PERIODS/SCHEMATICS/{fig_name}', dpi=600, transparent=True, bbox_inches='tight')

In [ ]:
fig_name = 'ACR_25__swi__OFF_INDUCTION_SCHEMATIC--EMG.png'
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = True
plt.rcParams['xtick.major.size'] = 14
plt.rcParams['axes.spines.left'] = False
plt.rcParams['axes.spines.right'] = False
plt.rcParams['ytick.left'] = False
for pulse in [pulse_to_start]:
    t1 = pon[pulse] - pd.Timedelta('60ms')
    t2 = t1 + pd.Timedelta(seconds=seconds)
    fmg, axmg = plt.subplots(1, 1, figsize=(18, 1.5))
    axmg.plot(emg.prb('EMGr').sel(datetime=slice(t1, t2)).ch(1).datetime, emg.prb('EMGr').sel(datetime=slice(t1, t2)).ch(1).data, color=EMG_SLATE, linewidth=6)
    axmg.set_xlim(t1, t2)
    axmg.set_xticks(xticks)

fmg.savefig(f'{PAPER_FIGURE_ROOT}/OFF_PERIODS/SCHEMATICS/{fig_name}', dpi=600, transparent=True, bbox_inches='tight')
axmg.get_ylim()

In [ ]:
fig_name = 'ACR_25__swi__OFF_INDUCTION_SCHEMATIC--NNXr-BASE.png'
plt.rcdefaults()
acr.plots.lrg()
plt.rcParams['xtick.bottom'] = True
plt.rcParams['xtick.major.size'] = 14
for pulse in [pulse_to_start]:
    t1 = pon[pulse] - pd.Timedelta('60ms')
    t2 = t1 + pd.Timedelta(seconds=seconds)
    # plot the optrode:
    lfp_dat = fp.sel(datetime=slice(t1, t2)).prb('NNXr').data.T
    lfp_times = fp.sel(datetime=slice(t1, t2)).prb('NNXr').datetime.values
    mua_dat = mua.ts(t1, t2).prb('NNXr')
    f, ax = kde.plot.main.plot_lfp_mua_combined(lfp_dat, mua_dat, lfp_times, lfp_subplot_hspace=-0.6, lfp_color=NNXR_GRAY, mua_color=NNXR_GRAY, figsize=FSIZE, mua_height_multiplier=mua_height_multiplier, lw=lw, rsz=rsz)
    xticks = [pd.Timestamp(lfp_times.min()) + pd.Timedelta(seconds=i) for i in range(seconds+1)]
    ax[16].set_xticklabels([])
    ax[16].set_xticks(xticks)
    for p in pon[pulse:pulse+6]:
        for i, a in enumerate(ax):
            if i != 16:
                a.set_ylim(-795, 785)
f.savefig(f'{PAPER_FIGURE_ROOT}/OFF_PERIODS/SCHEMATICS/{fig_name}', dpi=600, transparent=True, bbox_inches='tight')